In [525]:
import pandas as pd

In [526]:
adults_data = pd.read_csv("adult.csv")

In [527]:
adults_data.head()

,Unnamed: 0,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


In [528]:
adults_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      32561 non-null  int64 
 1   age             32561 non-null  int64 
 2   workclass       32561 non-null  object
 3   education       32561 non-null  object
 4   marital-status  32561 non-null  object
 5   occupation      32561 non-null  object
 6   relationship    32561 non-null  object
 7   race            32561 non-null  object
 8   sex             32561 non-null  object
 9   hours-per-week  32561 non-null  int64 
 10  native-country  32561 non-null  object
 11  salary          32561 non-null  object
 12  salary K$       32561 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 3.2+ MB


## Data preparation

In [529]:
adults_data.drop(columns=["Unnamed: 0"], inplace=True)

#### Remove rows with `?` value in any column. The number of rows reduced from **32561** to **30162**.

In [530]:
adults_data.astype(str).apply(lambda row: row.str.contains("\\?")).any(
    axis=1
).sum()

np.int64(2399)

In [531]:
mask_queation_mark = (
    adults_data.astype(str)
    .apply(lambda row: row.str.contains("\\?"))
    .any(axis=1)
)

In [532]:
adults_data = adults_data[~mask_queation_mark]

In [533]:
adults_data.shape

(30162, 12)

In [534]:
adults_data.astype(str).apply(lambda row: row.str.contains("\\?")).any(
    axis=1
).any().sum()

np.int64(0)

#### Columns with salary info preparation

In [535]:
adults_data.salary.isnull().sum()

np.int64(0)

In [536]:
adults_data.salary.value_counts()

salary
<=50K    22654
>50K      7508
Name: count, dtype: int64

* more suitable name set for `salary K$` table

In [537]:
adults_data.rename(columns={"salary K$": "salary_USD_k"}, inplace=True)

In [538]:
adults_data.columns.isin(["salary_USD_k"]).any()

np.True_

* implementation of **more_than_50k** with `boolean dtype` instead of dropped **salary** column

In [539]:
adults_data["more_than_50k"] = adults_data.salary.apply(
    lambda x: False if str(x).startswith("<=") else True
)

In [540]:
adults_data.more_than_50k.value_counts()

more_than_50k
False    22654
True      7508
Name: count, dtype: int64

In [541]:
adults_data.drop(columns=["salary"], inplace=True)

In [542]:
adults_data.head()

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary_USD_k,more_than_50k
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,39,False
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,35,False
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,27,False
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,43,False
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,25,False


* reset index after dropping rows

In [543]:
adults_data.reset_index(drop=True, inplace=True)

In [544]:
adults_data.index

RangeIndex(start=0, stop=30162, step=1)

* validating data in `salary_USD_k` and `more_than_50k` columns

In [545]:
adults_data[
    ((adults_data.more_than_50k == True) & (adults_data.salary_USD_k <= 50))
    | ((adults_data.more_than_50k == False) & (adults_data.salary_USD_k > 50))
]

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary_USD_k,more_than_50k


* implementing **categorial** columns

In [546]:
adults_data.workclass = adults_data.workclass.astype("category")
adults_data.relationship = adults_data.relationship.astype("category")
adults_data.sex = adults_data.sex.astype("category")
adults_data.race = adults_data.race.astype("category")
adults_data.education = adults_data.education.astype("category")
adults_data.occupation = adults_data.occupation.astype("category")
adults_data["marital-status"] = adults_data["marital-status"].astype(
    "category"
)

In [547]:
adults_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30162 entries, 0 to 30161
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   age             30162 non-null  int64   
 1   workclass       30162 non-null  category
 2   education       30162 non-null  category
 3   marital-status  30162 non-null  category
 4   occupation      30162 non-null  category
 5   relationship    30162 non-null  category
 6   race            30162 non-null  category
 7   sex             30162 non-null  category
 8   hours-per-week  30162 non-null  int64   
 9   native-country  30162 non-null  object  
 10  salary_USD_k    30162 non-null  int64   
 11  more_than_50k   30162 non-null  bool    
dtypes: bool(1), category(7), int64(3), object(1)
memory usage: 1.2+ MB


## Analysis

* sex exploring data in dataset

In [548]:
adults_data.sex.value_counts()

sex
Male      20380
Female     9782
Name: count, dtype: int64

In [549]:
f'Number of males and females: {int(adults_data.sex[adults_data.sex == "Male"].count())} and {int(adults_data.sex[adults_data.sex == "Female"].count())} respectively.'

'Number of males and females: 20380 and 9782 respectively.'

In [550]:
f'Average age of male in dataset is {adults_data.age[adults_data.sex == "Male"].mean().round(2)}.'

'Average age of male in dataset is 39.18.'

* calculation of people from Poland 

In [551]:
poles = adults_data[adults_data["native-country"] == "Poland"][
    "native-country"
].count()
all = adults_data["native-country"].count()
f"Percentage of people from Poland (total: {poles}) in dataset is {round(poles/all*100, 2)}%."

'Percentage of people from Poland (total: 56) in dataset is 0.19%.'

* people without higher **education** (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with a `salary > 50K`.

In [552]:
more_than_50k_wit_no_high_edu = adults_data[
    ~adults_data.education.isin(
        [
            "Bachelors",
            "Bachelors",
            "Prof-school",
            "Assoc-acdm",
            "Assoc-voc",
            "Masters",
            "Doctorate",
        ]
    )
    & (adults_data.more_than_50k == True)
]

In [553]:
f"People without higher education and with a salary > 50K count: {more_than_50k_wit_no_high_edu.shape[0]}."

'People without higher education and with a salary > 50K count: 3178.'

* age statistics for each type of education

In [554]:
adults_data.groupby("education")["age"].describe()

/var/folders/76/p9ls0c8576d26f9v9n864tq80000gn/T/ipykernel_60883/2844832061.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  adults_data.groupby("education")["age"].describe()


,count,mean,std,min,25%,50%,75%,max
education,,,,,,,,
10th,820.0,37.897561,16.225795,17.0,23.0,36.0,52.0,90.0
11th,1048.0,32.363550,15.089307,17.0,18.0,28.5,43.0,90.0
12th,377.0,32.013263,14.373710,17.0,19.0,28.0,41.0,79.0
1st-4th,151.0,44.622517,14.929051,19.0,33.0,44.0,56.0,81.0
5th-6th,288.0,41.649306,14.754622,17.0,28.0,41.0,53.0,82.0
7th-8th,557.0,47.631957,15.737479,17.0,34.0,49.0,60.0,90.0
9th,455.0,40.303297,15.335754,17.0,28.0,38.0,53.0,90.0
Assoc-acdm,1008.0,37.286706,10.509755,19.0,29.0,36.0,44.0,90.0
Assoc-voc,1307.0,38.246366,11.181253,19.0,30.0,37.0,45.0,84.0


* salaries of married and unmarried men

In [555]:
men_salaries_df = adults_data[adults_data.sex == "Male"].copy()

In [556]:
men_salaries_df.reset_index(drop=True, inplace=True)

In [557]:
men_salaries_df.loc[:, "married"] = (
    men_salaries_df["marital-status"]
    .str.startswith("Married")
    .apply(lambda x: "Married" if x else "Not married")
)

In [558]:
pd.pivot_table(
    men_salaries_df,
    values="salary_USD_k",
    index="more_than_50k",
    columns="married",
    aggfunc=["count"],
)

count            
married       Married Not married
more_than_50k                    
False            7052        6932
True             5723         673

* max working hourse and count of people working this amount of time per week

In [559]:
adults_data[
    adults_data["hours-per-week"] == int(adults_data["hours-per-week"].max())
].pivot_table(values="sex", index="hours-per-week", aggfunc=["count"]).rename(
    columns={"sex": "people_count"}
)

,count
,people_count
hours-per-week,
99,78


In [562]:
pd.pivot_table(
    adults_data,
    index="sex",
    values="hours-per-week",
    aggfunc=["max", "min", "mean"],
    columns="more_than_50k",
).rename(
    columns={
        "max": "max_hours_per_week",
        "min": "min_hours_per_week",
        "mean": "mean_hours_per_week",
        True: "more_than_50k",
        False: "less_than_50k",
    }
)

/var/folders/76/p9ls0c8576d26f9v9n864tq80000gn/T/ipykernel_60883/3103655589.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(
/var/folders/76/p9ls0c8576d26f9v9n864tq80000gn/T/ipykernel_60883/3103655589.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(
/var/folders/76/p9ls0c8576d26f9v9n864tq80000gn/T/ipykernel_60883/3103655589.py:1: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  pd.pivot_table(


max_hours_per_week               min_hours_per_week  \
more_than_50k      less_than_50k more_than_50k      less_than_50k   
sex                                                                 
Female                        99            99                  1   
Male                          99            99                  1   

                            mean_hours_per_week                
more_than_50k more_than_50k       less_than_50k more_than_50k  
sex                                                            
Female                    2           36.423645     40.897482  
Male                      1           41.162042     46.542683